# Building Custom MCP Servers

Create your own MCP tool server for quality control defect logging.

In [9]:
import os, subprocess, sys, time, threading, re, json
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime

subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "python-dotenv", "fastmcp", "azure-ai-projects==1.0.0", "azure-ai-agents==1.2.0b4", "azure-identity"])
load_dotenv('.env')
print("✓ Dependencies installed")

✓ Dependencies installed



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## 1. Business Logic - Quality Management System
In-memory defect tracking.

In [10]:
class QualityManagementSystem:
    def __init__(self):
        self.defects = []
        self.defect_counter = 1000
    
    def log_defect(self, description: str, severity: str = "medium"):
        defect_id = f"DEF-{self.defect_counter}"
        self.defect_counter += 1
        defect = {"defect_id": defect_id, "timestamp": datetime.now().isoformat(), "description": description, "severity": severity, "status": "open"}
        self.defects.append(defect)
        return defect
    
    def get_stats(self):
        return {"total": len(self.defects), "open": len([d for d in self.defects if d["status"] == "open"])}

qms = QualityManagementSystem()
print("✓ QMS initialized")

✓ QMS initialized


## 2. MCP Server - Expose Tools
Wrap business logic as MCP tools.

In [11]:
from fastmcp import FastMCP

mcp = FastMCP(name="quality-control")

@mcp.tool()
def log_defect(description: str, severity: str = "medium") -> dict:
    """Log manufacturing defect. Args: description (str), severity (low/medium/high)"""
    if severity not in ["low", "medium", "high"]:
        return {"success": False, "error": "Invalid severity"}
    defect = qms.log_defect(description, severity)
    return {"success": True, "defect_id": defect["defect_id"], "message": f"Defect {defect['defect_id']} logged"}

@mcp.tool()
def get_defect_summary() -> dict:
    """Get defect statistics and recent entries"""
    stats = qms.get_stats()
    return {"statistics": stats, "recent": qms.defects[-5:], "message": f"{stats['total']} defects, {stats['open']} open"}

print("✓ MCP server created with log_defect, get_defect_summary")

✓ MCP server created with log_defect, get_defect_summary


## 3. Start Local Server + Azure Dev Tunnel
Expose MCP server publicly.

In [ ]:
server_port = 8765

def run_server():
    mcp.run(transport="http", port=server_port, host="127.0.0.1")

threading.Thread(target=run_server, daemon=True).start()
time.sleep(2)
print(f"✓ MCP server running on http://127.0.0.1:{server_port}/mcp")

# Create Azure Dev Tunnel
host_process = subprocess.Popen(
    ["/home/vscode/bin/devtunnel", "host", "-p", str(server_port), "--allow-anonymous"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

start_time, public_url = time.time(), None
while True:
    line = host_process.stdout.readline()
    if not line:
        time.sleep(0.1)
        if time.time() - start_time > 30: break
        continue
    match = re.search(r'https://[a-z0-9\-]+(?:\.[a-z0-9\-]+)?\.devtunnels\.ms', line)
    if match:
        public_url = match.group(0)
        break

mcp_endpoint = f"{public_url}/mcp"
print(f"✓ Public MCP endpoint: {mcp_endpoint}")

## 4. Connect Azure AI Agent to MCP Server
Agent discovers tools automatically.

In [13]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import McpTool, SubmitToolApprovalAction, RequiredMcpToolCall, ToolApproval, ListSortOrder

project_client = AIProjectClient(endpoint=os.environ["PROJECT_ENDPOINT"], credential=DefaultAzureCredential())

mcp_tool = McpTool(server_label="quality_control", server_url=mcp_endpoint, allowed_tools=["log_defect", "get_defect_summary"])

agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="qc-agent",
    instructions="Help operators log defects. Extract description/severity. Severity: low (cosmetic), medium (functional), high (critical).",
    tools=mcp_tool.definitions
)

print(f"✓ Agent {agent.id} connected to MCP server")

✓ Agent asst_uLEtpyt8EbzFMZbyMaUsDJw3 connected to MCP server


## 5. Test Agent - Log Defect via Natural Language

In [14]:
thread = project_client.agents.threads.create()

project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Record defect: Paint coating uneven on left panel side, affecting finish quality."
)

run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    
    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        approvals = [ToolApproval(tool_call_id=tc.id, approve=True, headers=mcp_tool.headers) 
                    for tc in run.required_action.submit_tool_approval.tool_calls 
                    if isinstance(tc, RequiredMcpToolCall)]
        if approvals:
            run = project_client.agents.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_approvals=approvals)

messages = list(project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING))

for msg in messages:
    role = "OPERATOR" if msg.role == "user" else "AGENT"
    if msg.text_messages:
        print(f"{role}: {msg.text_messages[-1].text.value}")

OPERATOR: Record defect: Paint coating uneven on left panel side, affecting finish quality.
AGENT: Defect logged:
- Description: Paint coating uneven on left panel side, affecting finish quality.
- Severity: Medium (functional issue).

Let me know if you need to log more defects or view defect summaries.


## 6. Get Summary Statistics

In [15]:
project_client.agents.messages.create(thread_id=thread.id, role="user", content="Show defect summary")

run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        approvals = [ToolApproval(tool_call_id=tc.id, approve=True, headers=mcp_tool.headers) 
                    for tc in run.required_action.submit_tool_approval.tool_calls if isinstance(tc, RequiredMcpToolCall)]
        if approvals:
            run = project_client.agents.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_approvals=approvals)

messages = list(project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING))
if messages[-1].role == "assistant" and messages[-1].text_messages:
    print(messages[-1].text_messages[-1].text.value)

Defect Summary:
- Total defects: 1
- Open defects: 1

Recent Defect:
- ID: DEF-1000
- Description: Paint coating uneven on left panel side, affecting finish quality.
- Severity: Medium
- Status: Open
- Logged: 2025-10-07

Let me know if you need details on this defect, wish to update a defect, or want to log another issue.


## Cleanup

In [16]:
project_client.close()
subprocess.run(["/home/vscode/bin/devtunnel", "delete-all"], capture_output=True, timeout=10)
print("✓ Cleanup complete")

✓ Cleanup complete


## Summary

**What We Built:**
1. **QMS** - Business logic (Python class)
2. **MCP Server** - Tool wrapper (FastMCP)
3. **Dev Tunnel** - Public endpoint (Azure Dev Tunnels)
4. **AI Agent** - Natural language interface (Azure AI Foundry)

**Key Concept:** Wrap any business logic as MCP tools. Agents discover and call them automatically.

**Benefits:**
- Decoupled: Update tools without changing agents
- Reusable: One MCP server serves multiple agents
- Natural: Operators use plain language, not forms